# 0. Install Dependencies

In [4]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Toy Environment with OpenAI Gym

In [23]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
from gym import spaces
# import pandas

In [11]:
vocab = ["", "I", "he", "she", "they", "am", "is", "are", "was", "were", "go", "going", "goes", "went", "market", 
         "amusement-park", "to", "a", "an", "the"]
vocab_size = len(vocab)
max_len = 10
Sample_sent = ["I is going to market.", "She am go to market.", "They are go to a amusement-park.", 
               "He going to an market.", "I were going."]

In [29]:
def print_sent(sent):
    table = []
    sent = sent.split()
    for i in range(len(sent)):
        print(sent[i]+"(" + str(i) + ")", end = " ")

In [25]:
print_sent("I am shreya")

I(0) am(1) shreya(2) 

In [46]:
class OurCustomEnv(Env):

    def __init__(self):

        self.sent = ""
        #self.sales_function = sales_function     #calculating sales based on spends
        self.high =np.array([vocab_size] * max_len, dtype=np.int32)
        self.low =np.array([0] * max_len, dtype=np.int32)
        
        #we create an observation space with predefined range
        self.observation_space = Box(low=self.low, high=self.high, dtype = np.float32)

        #similar to observation, we define action space 
        self.action_space = ["Replace", "Delete", "Add"]
        
    
    def step(self, action, arg1, arg2=0):
        
        if action == "Delete":
            sent1 = self.sent.split()
            self.sent = " ".join(sent1[:arg1] + sent1[arg1+1:])
        
        elif action == "Add":
            sent1 = self.sent.split()
            self.sent = " ".join(sent1[:arg1] + [vocab[arg2]] + sent1[arg1:])
            
        elif action == "Replace":
            sent1 = self.sent.split()
            self.sent = " ".join(sent1[:arg1] + [vocab[arg2]] + sent1[arg1+1:])
        
        reward = 0.0
        reward = 1  #gives total sales based on spends
        done = True                            #Condition for completion of episode
        info = {}        

        return self.sent, reward, done, info 

    def reset(self):
        self.sent = Sample_sent[random.randint(0, len(Sample_sent))-1]
        return self.sent
    
def act(action, ind1, string=""):
    if action == "Delete":
        return env.step("Delete", ind1)
    else:
        return env.step(action, ind1, vocab.index(string))    

In [56]:
env = OurCustomEnv()
state = env.reset()

In [57]:
print_sent(state)

She(0) am(1) go(2) to(3) market.(4) 

In [58]:
act1 = act("Delete", 1)
act1

('She go to market.', 1, True, {})

In [59]:
act2 = act("Replace", 1, "goes")
act2

('She goes to market.', 1, True, {})

In [60]:
act3 = act("Add", 3, "the")
act3

('She goes to the market.', 1, True, {})